# 하드보팅

In [170]:
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp
import cudf
from cuml.model_selection import train_test_split

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


In [171]:
df_dh = cudf.read_csv("process_dohun.csv")
df_dh = df_dh.drop('subject_id', axis=1)
df_dh = df_dh.drop('hadm_id', axis=1)
df_dh


y = df_dh['hospital_expire_flag'] 
X = df_dh.drop('hospital_expire_flag', axis=1) # 'label' 열을 제외한 나머지 열을 X로 사용


In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

X_train = X_train.to_cupy()
X_test = X_test.to_cupy()
y_train = y_train.to_cupy()
y_test = y_test.to_cupy()

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [173]:
model1 = pd.read_csv('xgboost_y_pred.csv',header=None)
# model2 = pd.read_csv('randomForest_y_pred.csv', header=None)
model3 = pd.read_csv('lightgbm_y_pred.csv',header=None)
model4 = pd.read_csv('adaboost_pred.csv',header=None)
model5 = pd.read_csv('catboost_y_pred.csv',header=None)


In [174]:
combined_predictions = pd.concat([model1, model3, model4, model5], axis=1)
print(combined_predictions)
# 다수결로 하드 보팅을 수행합니다.
hard_voting = combined_predictions.mode(axis=1).iloc[:, 0]


# 하드 보팅 결과를 출력합니다.
print("하드 보팅 결과:")
print(hard_voting)

         0    0    0    0
0        1  1.0  1.0  1.0
1        1  1.0  1.0  1.0
2        1  1.0  1.0  1.0
3        1  1.0  1.0  1.0
4        1  1.0  1.0  1.0
...     ..  ...  ...  ...
2070055  0  0.0  0.0  0.0
2070056  0  0.0  0.0  0.0
2070057  0  0.0  0.0  0.0
2070058  0  0.0  0.0  0.0
2070059  1  1.0  1.0  1.0

[2070060 rows x 4 columns]
하드 보팅 결과:
0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
2070055    0.0
2070056    0.0
2070057    0.0
2070058    0.0
2070059    1.0
Name: 0, Length: 2070060, dtype: float64


In [175]:
print(y_test)

[1. 0. 1. ... 0. 0. 1.]


In [176]:
# import cuml
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


# X_train = X_train.to_cupy()
# X_test = X_test.to_cupy()
# y_train = y_train.to_cupy()
# y_test = y_test.to_cupy()

# cu_score = cuml.metrics.accuracy_score(y_test, hard_voting )
# print(cu_score)

# 정확도 계산
accuracy = accuracy_score(y_test.get(), hard_voting)
print("Accuracy:", accuracy)

# F1 스코어 계산
f1 = f1_score(y_test.get(), hard_voting)
print("F1 Score:", f1)

Accuracy: 0.9271161222380028
F1 Score: 0.927467840457517
